# Import Models as Python Functions

A useful feature of yggdrasil is that you can use the mechanics to call models as functions from inside Python even if the model is not written in Python. For example, the model displayed below is written in Fortran, but can be called from Python via the yggdrasil `import_as_function` method.

In [1]:
from yggdrasil import tools
tools.display_source('models/light.f90', number_lines=True)
tools.display_source('yamls/light_fortran.yml', number_lines=True)

file: models/light.f90
 1: function light(height, time, intensity) result(out)
 2:   real(kind=8), dimension(:), intent(in) :: height
 3:   real(kind=8), intent(in) :: time
 4:   real(kind=8), dimension(:), allocatable :: intensity
 5:   logical :: out
 6:   integer :: i
 7:   real, parameter :: Pi = 3.1415927
 8:   out = .true.
 9:   if (allocated(intensity)) then
10:      deallocate(intensity)
11:   end if
12:   allocate(intensity(size(height)))
13:   do i = 1, size(height)
14:      intensity(i) = 80.0 * (1.0 + SIN(2.0 * Pi * time / 365)) / (ABS(200.0 - height(i))**2)
15:   end do
16: end function light

file: yamls/light_fortran.yml
 1: model:
 2:   name: light
 3:   language: fortran
 4:   args: ../models/light.f90
 5:   function: light
 6:   is_server: True
 7:   outputs: [intensity]



When import_as_function is called, the model yaml is loaded and models contained in the yaml are run on forked processes after being copiled as necessary. 

In [2]:
from yggdrasil import import_as_function
light = import_as_function('yamls/light_fortran.yml')

INFO:runner.startDrivers[481]:YggRunner(runner): Starting I/O drivers and models on system None in namespace yggdrasil with rank 0
/Users/langmm/yggdrasil/yggdrasil/demos/CiS2021-hackathon/models/ygg_light_f90_gfortranx_gfortranx.out
INFO:runner.run[368]:YggRunner(runner):                 init	0.000001
INFO:runner.run[368]:YggRunner(runner):         load drivers	0.482005
INFO:runner.run[368]:YggRunner(runner):        start drivers	0.073659
INFO:runner.run[370]:YggRunner(runner): ========================================
INFO:runner.run[371]:YggRunner(runner):                Total	0.555665


The returned function's inputs & outputs are determined by the unpaired inputs/outputs located in the yaml. In this example, the only inputs & outputs come from the server.

In [3]:
light.info()

Models: light
Inputs:
	light (vars=['height', 'time'])
Outputs:
	light (vars=['intensity'])



In [4]:
import numpy as np
print(light(np.arange(10, dtype='f8'), 100.0))
print(light(np.arange(100, dtype='f8'), 100.0))
# print(light(100.0, 100.0))

{'intensity': array([0.00397736, 0.00401743, 0.00405811, 0.00409942, 0.00414135,
       0.00418394, 0.00422718, 0.0042711 , 0.00431571, 0.00436102])}
{'intensity': array([0.00397736, 0.00401743, 0.00405811, 0.00409942, 0.00414135,
       0.00418394, 0.00422718, 0.0042711 , 0.00431571, 0.00436102,
       0.00440704, 0.0044538 , 0.00450131, 0.00454958, 0.00459863,
       0.00464848, 0.00469914, 0.00475064, 0.00480299, 0.00485621,
       0.00491032, 0.00496533, 0.00502128, 0.00507818, 0.00513605,
       0.00519491, 0.0052548 , 0.00531572, 0.00537771, 0.00544079,
       0.00550499, 0.00557033, 0.00563684, 0.00570455, 0.00577349,
       0.00584368, 0.00591516, 0.00598796, 0.00606212, 0.00613766,
       0.00621462, 0.00629303, 0.00637295, 0.00645439, 0.0065374 ,
       0.00662203, 0.00670831, 0.00679628, 0.006886  , 0.00697751,
       0.00707085, 0.00716608, 0.00726325, 0.0073624 , 0.00746361,
       0.00756691, 0.00767237, 0.00778005, 0.00789001, 0.00800232,
       0.00811705, 0.00823426, 0

In [5]:
light.stop()

End of input from [yggarg(height_realloc), yggarg(time)].
INFO:runner.waitModels[538]:YggRunner(runner): light finished running.
INFO:runner.waitModels[544]:YggRunner(runner): light finished exiting.
INFO:runner.waitModels[538]:YggRunner(runner): function_model finished running.
INFO:runner.waitModels[544]:YggRunner(runner): function_model finished exiting.
INFO:runner.waitModels[558]:YggRunner(runner): All models completed
